In [1]:
!pip uninstall opencv-python

Found existing installation: opencv-python 4.1.2.30
Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-python-4.1.2.30


In [3]:
!pip uninstall opencv-contrib-python

In [4]:
!pip install opencv-contrib-python

     |████████████████████████████████| 66.7 MB 20 kB/s 


##Importing required libraries

In [5]:
import cv2
from matplotlib import pyplot as plt
from pylab import *
from numpy import *
from scipy import linalg
import pandas as pd
import numpy as np

In [6]:
K = [[1698.873755, 0.000000, 971.7497705],
     [0.000000, 1698.8796645, 647.7488275],
     [0.000000, 0.000000, 1.000000]]

K = np.asarray(K)

In [7]:
K_inverse = np.linalg.inv(K)

In [8]:
K

array([[1.69887376e+03, 0.00000000e+00, 9.71749770e+02],
       [0.00000000e+00, 1.69887966e+03, 6.47748827e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [9]:
K_inverse

array([[ 5.88625257e-04,  0.00000000e+00, -5.71996458e-01],
       [ 0.00000000e+00,  5.88623209e-04, -3.81279994e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [10]:
image_1 = cv2.imread('/content/im1.jpg', 0)
image_2 = cv2.imread('/content/im2.jpg', 0)

In [17]:
def find_ground_truth_correspondences(image1, image2):
  #Class for extracting keypoints and computing descriptors using the Scale Invariant Feature Transform
  sift = cv2.xfeatures2d.SIFT_create()
  #find keypoints and descriptors
  kp1, des1 = sift.detectAndCompute(image1,None)
  kp2, des2 = sift.detectAndCompute(image2,None)
  #Flann-based descriptor matcher.
  flann = cv2.FlannBasedMatcher(dict(algorithm = 0, trees = 5),dict(checks=50))
  matches = flann.knnMatch(des1,des2,k=2)
  src_pts= []
  dst_pts = []

  # store all the good matches as per Lowe's ratio test.
  j = 0
  for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        dst_pts.append((kp2[m.trainIdx].pt, 1))
        src_pts.append(kp1[m.queryIdx].pt)
        j = j+1
        if j == 100:
          break
  src_pts = [list(tup)+[1] for tup in src_pts]
  dst_pts = [list(tup)+[1] for tup in dst_pts]
  src_pts, pts2 = np.asarray(src_pts), np.asarray(dst_pts)
  src_pts = np.float64(src_pts)
  dst_pts = np.float64(src_pts)

  return src_pts, dst_pts

In [18]:
src_pts, dst_pts = find_ground_truth_correspondences(image_1, image_2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [19]:
def convert_world_to_camera_coordinate_system(src_pts, dst_pts, k):
  k_inv = np.linalg.inv(k)
  src_pts = [np.dot(k_inv, item) for item in src_pts]
  dst_pts = [np.dot(k_inv, item) for item in dst_pts]
  return src_pts, dst_pts

In [20]:
def build_matrix_for_eq(src_pts,dst_pts):
  A = []
  for i in range(8):
    z = np.kron(dst_pts[i], src_pts[i])
    A.append(z)
  A = np.asarray(A)
  print(A.shape)
  return A

In [21]:
def compute_essential(src_pts, dst_pts):
  
  x1, x2 = convert_world_to_camera_coordinate_system(src_pts, dst_pts, K)
  x1 = np.asarray(x1)
  x2 = np.asarray(x2)
  n = x1.shape[1]
  if x2.shape[1] != n:
    raise ValueError("Number of points don't match.")
 
  A = build_matrix_for_eq(src_pts, dst_pts)


  U,S,V = linalg.svd(A)
  E = V[-1].reshape(3,3)
  # constrain F
  # make rank 2 by zeroing out last singular value
  Ua,Sa,Va = linalg.svd(E)
  
  sigma = (Sa[0]+Sa[1])/2
  Sa[0] = sigma
  Sa[1] = sigma
  Sa[2] = 0
  
  Ea = np.dot(Ua,np.dot(diag(Sa),Va))

  return sigma , Ea

In [22]:
sigma, E = compute_essential(src_pts, dst_pts)

(8, 9)


In [23]:
E

array([[-3.43233757e-22,  1.80399143e-08, -3.09152220e-06],
       [-1.80399142e-08, -1.65426508e-18,  7.07106781e-01],
       [ 3.09152221e-06, -7.07106781e-01, -7.18875573e-20]])

In [24]:
def compute_Rotation_and_Translation_matrix(E, sigma):
  #Singular Value Decomposition
  U,S,V = linalg.svd(E)

  R_z = [[0, -1, 0],
         [1, 0, 0],
         [0, 0, 1]]
  R_z = np.asarray(R_z)
             
  R1 = np.dot(U ,np.dot(R_z, V))
  R2 = np.dot(U ,np.dot(np.transpose(R_z), V))
  T1 = U[:,2]
  T2 = -U[:,2]

  return R1, R2, T1, T2

In [25]:
R1, R2, T1, T2 = compute_Rotation_and_Translation_matrix(E, sigma)

In [27]:
def compute_M_and_N_matrix(R1, R2, T1, T2,K):
  I_O = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]]
  M = np.dot(K, I_O)
  T1 = np.expand_dims(T1, axis = 1)
  R_t = np.hstack((R1,T1))
  N = np.dot(K, R_t)
  return M, N

In [28]:
M, N = compute_M_and_N_matrix(R1, R2, T1, T2,K)

Let Pi be the corresponding 3D point for the pixel pair (pi
, p
0
i
). Find Pi
, ∀i ∈ {1, 2, . . . , n} using

the triangulation approach learned in the class.

In [29]:
def triangulate_point(X1,X2,P1,P2):    
    A = zeros((6,6))
    A[:3,:4] = P1
    A[3:,:4] = P2
    A[:3,4] = -X1
    A[3:,5] = -X2
    U,S,V = linalg.svd(A)
    X = V[-1,:4]
    return X / X[3]

In [30]:
def triangulation_approach(pts1,pts2,P1,P2):   
    n = pts1[0].reshape(3,1).shape[1]
    three_d_points = []    
    for j in range(len(pts1)):
      X1 = pts1[j].reshape(3,1)
      X2 = pts2[j].reshape(3,1)
      X = [ triangulate_point(X1[:,i],X2[:,i],P1,P2) for i in range(n)]
      three_d_points.append(array(X).T)
    return three_d_points

In [31]:
ff = triangulation_approach(src_pts,dst_pts,M,N)

In [32]:
ff

[array([[-4.25704011e+15],
        [ 4.84316949e+14],
        [ 1.01021714e+16],
        [ 1.00000000e+00]]), array([[ 4.74624336e+14],
        [-5.19344520e+14],
        [-2.16635194e+15],
        [ 1.00000000e+00]]), array([[ 1.40972719e+15],
        [-1.62119964e+15],
        [-6.53980582e+15],
        [ 1.00000000e+00]]), array([[ 3.69972884e+14],
        [-3.97056600e+14],
        [-1.72343128e+15],
        [ 1.00000000e+00]]), array([[-6.39996796e+15],
        [ 7.81561073e+15],
        [ 3.04056874e+16],
        [ 1.00000000e+00]]), array([[ 1.18582646e+15],
        [-1.26985822e+15],
        [-5.64991354e+15],
        [ 1.00000000e+00]]), array([[-2.34089716e+15],
        [ 3.06047920e+15],
        [ 1.12588756e+16],
        [ 1.00000000e+00]]), array([[ 1.13417963e+15],
        [-1.29536839e+15],
        [-5.45720188e+15],
        [ 1.00000000e+00]]), array([[ 5.55997845e+14],
        [-7.42289119e+14],
        [-2.68641434e+15],
        [ 1.00000000e+00]]), array([[ 2.7445486